In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from yellowbrick.regressor import PredictionError
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%%time
ticket_scans = pd.read_csv('Data/Ticket Scan Data.csv')
game_data = pd.read_csv('Data/Game Data.csv')
team_data = pd.read_csv('Data/Team Data.csv')
section_data = pd.read_csv('Data/Seating Chart.csv')

Wall time: 3.02 s


In [3]:
ticket_scans.head()

,scan_unique_id,event_name,event_datetime,section_name,row_name,seat_number,scan_gate,scan_location,scan_datetime
0,124908643:19208870,LA Clippers 2018-2019 Game 14 - Dallas - 12/20/18,2018-12-20 19:30:00,332,8,23,11th Street - Door 1,STAPLES Center (Premium),2018-12-20 20:50:45
1,124908643:19309978,LA Clippers 2018-2019 Game 23 - Utah - 01/16/19,2019-01-16 19:30:00,304,8,16,11th Street - Door 2,STAPLES Center (Premium),2019-01-16 21:02:08
2,124908643:19198830,LA Clippers 2018-2019 Game 13 - Portland - 12/...,2018-12-17 19:30:00,207,10,15,Star Plaza - Door 1,STAPLES Center (Premium),2018-12-17 20:54:52
3,124908643:19233811,LA Clippers 2018-2019 Game 16 - Sacramento - 1...,2018-12-26 19:30:00,107,D,6,Figueroa - Door 4,STAPLES Center (Premium),2018-12-26 21:07:30
4,124908643:19215364,LA Clippers 2018-2019 Game 14 - Dallas - 12/20/18,2018-12-20 19:30:00,107,10,13,Star Plaza - Door 6,STAPLES Center (Premium),2018-12-20 21:55:18


In [ ]:
ticket_scans['scan_datetime'] = pd.to_datetime(ticket_scans['scan_datetime'])
ticket_scans['event_datetime'] = pd.to_datetime(ticket_scans['event_datetime'])
ticket_scans['time_diff'] = (ticket_scans['scan_datetime'] - ticket_scans['event_datetime']).progress_apply(lambda x: int(x.total_seconds() //60))
def get_timebin(x):
    if x >= 0:
        return f'{(x//15+1)*15}-{x//15*15} Before'
    else:
        x = -1 * x
        return f'{(x//15+1)*15}-{x//15*15} After'
    
ticket_scans['time_diff'] = ticket_scans['time_diff'].progress_apply(get_timebin)

In [ ]:
time_period = ticket_scans['time_diff'].unique().tolist()

In [ ]:
ticket_scans = pd.get_dummies(ticket_scans, columns=['time_diff'])

In [ ]:
ticket_scans.columns = [column.replace('time_diff_', '') for column in ticket_scans.columns]
ticket_scans['scan_unique_id'] = 1

In [ ]:
ticket_scans.head()

In [ ]:
%%time
ticket_scans['City'] = ticket_scans['event_name'].progress_apply(lambda x: x.split('-')[-2][1:-1])
ticket_scans['event_datetime'] = ticket_scans['event_datetime'].dt.normalize()
ticket_scans = ticket_scans.groupby(['City', 'event_datetime', 'section_name'])[time_period+['scan_unique_id']].sum().reset_index().rename(columns = {'scan_unique_id':'attendees'})
ticket_scans.head()

In [ ]:
section_names = list(map(lambda x: str(x), ticket_scans['section_name'].unique()))

In [ ]:
%%time
section_dict = {}
for i in tqdm(range(59)):
    section_dict[int(ticket_scans['section_name'].tolist()[i::59][0])] = ticket_scans['attendees'].tolist()[i::59]

section_dict = pd.DataFrame(section_dict)
section_dict.head()

In [ ]:
%%time
ticket_scans = ticket_scans.groupby(['City', 'event_datetime'])[time_period+['attendees']].sum().reset_index()
for key in tqdm(section_dict.columns):
    ticket_scans[key] = section_dict[key]
ticket_scans.head()

In [ ]:
%%time
ticket_scans['City'] = ticket_scans['City'].replace('Golden State', 'Oakland')

In [ ]:
%%time
df = ticket_scans.merge(team_data[team_data['Team Full Name'] != 'Los Angeles Clippers'], on = 'City', how = 'left')
df['event_datetime'] = pd.to_datetime(df['event_datetime'])
df.head()

In [ ]:
df.columns = list(map(lambda x: str(x), df.keys()))

In [ ]:
%%time
game_data['Date'] = pd.to_datetime(game_data['Date'], format  = '%a, %b %d, %Y')

In [ ]:
df = df.merge(game_data, left_on = ['event_datetime', 'Team Full Name'], right_on = ['Date', 'Opponent'], how = 'left')

In [ ]:
df.isna().sum()[df.isna().sum()>0]

In [ ]:
section = section_data.groupby('section_name')['seat_count'].sum().reset_index().transpose()
section.columns = section.iloc[0]
section = section.drop(section.index[0])

In [ ]:
for key in tqdm(section.keys()):
    if key in df.keys():
        df[key] = df[key] / section[key][0] * 100

In [ ]:
df.head()

In [ ]:
%%time
df['week_day'] = df['event_datetime'].dt.day_name()
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_day = df.groupby('week_day')['attendees'].mean().reindex(cats).reset_index()
weekends = week_day[week_day['week_day'].isin(['Friday', 'Saturday', 'Sunday'])]

In [ ]:
df['is_weekend'] = df['week_day'].apply(lambda x: 'Weekend' if x in ['Friday', 'Saturday', 'Sunday'] else 'Weekday')
weekend = df.groupby('is_weekend')['attendees'].mean().reset_index()
print(weekend)

In [ ]:
plt.figure(figsize = (10,8))
bar = plt.bar(weekend['is_weekend'], weekend['attendees'])
bar[1].set_color('r')
plt.axhline(y = weekend['attendees'].mean(), color = 'grey', linestyle = '--', linewidth = 1.5)
plt.title('Average Attendance by Whether its a Weekend or not')
plt.ylabel('Average Attendance')
plt.xlabel('Weekend or Weekday')
plt.plot()

In [ ]:
plt.figure(figsize = (10,8))
ax = plt.gca()
plt.bar(week_day['week_day'], week_day['attendees'], label = 'Weekday')
plt.bar(weekends['week_day'], weekends['attendees'], color = 'red', label = 'Weekend')
plt.axhline(y = 8356.272727, color = 'grey', linestyle = '-.', linewidth = 2, label = 'Weekday Average')
plt.axhline(y = 8507.105263, color = 'grey', linestyle = ':', linewidth = 2, label = 'Weekend Average')
plt.title('Average Attendance by Day of Week')
plt.ylabel('Average Attendance')
plt.xlabel('Day of Week')
plt.legend()
plt.show()

In [ ]:
df.head()

In [ ]:
top4_attendance = df.groupby('Opponent')['attendees'].mean().reset_index().sort_values(by = 'attendees', ascending = False).head(4)
top4_attendance = top4_attendance.reset_index(drop = True)
top4_attendance

In [ ]:
plt.figure(figsize = (10,8))
plt.bar(top4_attendance['Opponent'], top4_attendance['attendees'], color = sns.color_palette('Set1'))
plt.axhline(y = top4_attendance['attendees'].mean(), color = 'grey', linestyle = ':', linewidth = 1.5)
plt.title('Top 4 Opponents with Highest Average Attendance')
plt.ylabel('Average Attendance')
plt.xlabel("Team played Against")
plt.show()

In [ ]:
top10_sections = df[section_names].mean().reset_index().sort_values(by = 0, ascending = False).head(10)
top10_sections = top10_sections.rename(columns = {'index': 'section_name', 0: 'percentage_filled'}).reset_index(drop = True)
top10_sections = top10_sections.reset_index().rename(columns = {'index': 'rank'})
top10_sections['rank'] = top10_sections['rank'] + 1
top10_sections

In [ ]:
plt.figure(figsize = (10,8))
plt.bar(top10_sections['section_name'], top10_sections['percentage_filled'], color = sns.color_palette("GnBu_d"))
plt.axhline(y = top10_sections['percentage_filled'].mean(), color = 'grey', linestyle = '--', linewidth = 1.5)
ax = plt.gca()
vals = ax.get_yticks()
vals = [str(int(val))+'%' for val in vals]
ax.set_yticklabels(vals)
plt.title('Top 10 Sections most filled to their capacity on an Average')
plt.ylabel('Average Percentage Filled')
plt.xlabel('Section Names')
plt.show()

In [ ]:
df.head()

In [ ]:
df['attendees'].corr(df['Vegas Odds for 1819'])

In [ ]:
plt.figure(figsize = (10,8))
plt.scatter(df['attendees'], df['Vegas Odds for 1819'])
plt.show()

In [ ]:
X = df[['Vegas Odds for 1819']]
y = df['attendees']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

plt.figure(figsize = (10,8))
ax = plt.gca()

model = LinearRegression(n_jobs = -1, normalize = True)
visualizer = PredictionError(model, ax = ax)
visualizer.fit(X_train, y_train)  
visualizer.score(X_test, y_test)  
visualizer.show()
plt.show()

In [ ]:
top10_time_period = df[time_period].mean().reset_index().rename(columns = {'index': 'time_period', 0: 'avg_number_of_scans'}).sort_values(by = 'avg_number_of_scans', ascending = False).head(10)
top10_time_period.head(1)

In [ ]:
plt.figure(figsize = (10, 8))
plt.bar(top10_time_period['time_period'], top10_time_period['avg_number_of_scans'], color = sns.color_palette("GnBu_d"))
plt.xticks(rotation=45)
plt.title('Average number of tickets scaned in 15 minute windows')
plt.xlabel('15 minute windows')
plt.ylabel('Average number of scans')
plt.show()